In [1]:
import os
from typing import List, Dict, Any, Annotated, Literal
from typing_extensions import TypedDict
import json
from datetime import datetime
import html

# LangChain and LangGraph imports
from langchain_core.messages import AnyMessage, HumanMessage, AIMessage, SystemMessage
from langchain_core.tools import tool
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

# Tool imports
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import ArxivAPIWrapper, WikipediaAPIWrapper
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

# Environment setup
from dotenv import load_dotenv
load_dotenv()



True

# ENHANCED STATE SCHEMA WITH MEMORY AND CONTEXT


In [2]:


# ================================================================================
# ENHANCED STATE SCHEMA WITH MEMORY AND CONTEXT
# ================================================================================

class ConversationState(TypedDict):
    """Enhanced state schema with conversation memory and context"""
    messages: Annotated[List[AnyMessage], add_messages]
    user_context: Dict[str, Any]  # Store user preferences, history, etc.
    tool_results_cache: Dict[str, Any]  # Cache recent tool results
    conversation_summary: str  # Summary of conversation so far
    error_count: int  # Track errors for fallback strategies
    last_tool_used: str  # Track last successful tool


# CUSTOM TOOLS


In [3]:

# ================================================================================
# CUSTOM TOOLS
# ================================================================================

@tool
def calculator(expression: str) -> str:
    """
    Perform mathematical calculations safely.
    
    Args:
        expression: Mathematical expression to evaluate (e.g., "2 + 3 * 4")
    
    Returns:
        Result of the calculation or error message
    """
    try:
        # Safe evaluation - only allow basic math operations
        allowed_chars = set('0123456789+-*/.() ')
        if not all(c in allowed_chars for c in expression):
            return "Error: Invalid characters in expression. Only numbers and basic operators (+, -, *, /, parentheses) are allowed."
        
        result = eval(expression)
        return f"Result: {result}"
    except Exception as e:
        return f"Error calculating '{expression}': {str(e)}"

@tool
def code_analyzer(code: str, language: str = "python") -> str:
    """
    Analyze code for basic syntax and provide simple feedback.
    
    Args:
        code: Code snippet to analyze
        language: Programming language (default: python)
    
    Returns:
        Basic analysis of the code
    """
    try:
        analysis = {
            "language": language,
            "lines": len(code.split('\n')),
            "characters": len(code),
            "contains_functions": "def " in code if language == "python" else "function " in code,
            "contains_classes": "class " in code if language == "python" else False,
            "contains_imports": any(line.strip().startswith(('import ', 'from ')) for line in code.split('\n')) if language == "python" else False
        }
        
        feedback = f"""
Code Analysis for {language}:
- Lines of code: {analysis['lines']}
- Total characters: {analysis['characters']}
- Contains functions: {analysis['contains_functions']}
- Contains classes: {analysis['contains_classes']}
- Contains imports: {analysis['contains_imports']}
        """
        
        if language == "python":
            try:
                compile(code, '<string>', 'exec')
                feedback += "\n- Syntax: Valid Python syntax"
            except SyntaxError as e:
                feedback += f"\n- Syntax Error: {str(e)}"
        
        return feedback.strip()
    except Exception as e:
        return f"Error analyzing code: {str(e)}"

@tool
def weather_info(location: str) -> str:
    """
    Get weather information for a location (simulated for demo).
    
    Args:
        location: City or location name
    
    Returns:
        Weather information
    """
    # Simulated weather data - in real implementation, use weather API
    import random
    
    weather_conditions = ["sunny", "cloudy", "rainy", "partly cloudy", "windy"]
    temperature = random.randint(15, 35)
    condition = random.choice(weather_conditions)
    
    return f"""
Weather for {location}:
- Temperature: {temperature}°C
- Condition: {condition.title()}
- Humidity: {random.randint(40, 80)}%
- Wind Speed: {random.randint(5, 25)} km/h
    """.strip()

@tool
def file_content_generator(file_type: str, content_description: str) -> str:
    """
    Generate sample file content based on type and description.
    
    Args:
        file_type: Type of file (e.g., 'csv', 'json', 'python', 'markdown')
        content_description: Description of what the file should contain
    
    Returns:
        Generated file content
    """
    try:
        if file_type.lower() == 'csv':
            return f"""# Sample CSV for: {content_description}
name,age,city,score
Alice,25,New York,85
Bob,30,London,92
Charlie,35,Tokyo,78
Diana,28,Paris,88"""

        elif file_type.lower() == 'json':
            sample_data = {
                "description": content_description,
                "data": [
                    {"id": 1, "name": "Item 1", "value": 100},
                    {"id": 2, "name": "Item 2", "value": 250},
                    {"id": 3, "name": "Item 3", "value": 175}
                ],
                "metadata": {
                    "created": datetime.now().isoformat(),
                    "version": "1.0"
                }
            }
            return json.dumps(sample_data, indent=2)

        elif file_type.lower() == 'python':
            return f'''"""
{content_description}
"""

def main():
    """Main function for {content_description}"""
    print("Hello, World!")
    
    # Add your code here
    data = [1, 2, 3, 4, 5]
    result = process_data(data)
    print(f"Result: {{result}}")

def process_data(data):
    """Process the input data"""
    return sum(data)

if __name__ == "__main__":
    main()
'''

        elif file_type.lower() == 'markdown':
            return f"""# {content_description}

## Overview
This document covers {content_description.lower()}.

## Key Points
- Point 1: Important information
- Point 2: Additional details
- Point 3: Summary notes

## Code Example
```python
def example_function():
    return "Hello, World!"
```

## Conclusion
This covers the basics of {content_description.lower()}.
"""

        else:
            return f"Sample content for {file_type} file:\n{content_description}\n\nGenerated on: {datetime.now()}"
            
    except Exception as e:
        return f"Error generating {file_type} content: {str(e)}"

# ================================================================================
# TOOL INITIALIZATION
# ================================================================================

def initialize_tools():
    """Initialize all available tools"""
    
    # Academic research tools
    arxiv_wrapper = ArxivAPIWrapper(top_k_results=3, doc_content_chars_max=1000)
    arxiv_tool = ArxivQueryRun(
        api_wrapper=arxiv_wrapper,
        description="Search academic papers on arXiv. Best for recent research papers and preprints."
    )
    
    # Wikipedia for general knowledge
    wiki_wrapper = WikipediaAPIWrapper(top_k_results=2, doc_content_chars_max=800)
    wiki_tool = WikipediaQueryRun(
        api_wrapper=wiki_wrapper,
        description="Search Wikipedia for general knowledge, definitions, and factual information."
    )
    
    # Web search tools
    tools_list = [arxiv_tool, wiki_tool]
    
    # Add Tavily if API key is available
    if os.getenv("TAVILY_API_KEY"):
        tavily_tool = TavilySearchResults(
            max_results=5,
            description="Search the web for recent news, current events, and real-time information."
        )
        tools_list.append(tavily_tool)
    
    # Add DuckDuckGo as fallback web search
    try:
        ddg_wrapper = DuckDuckGoSearchAPIWrapper(max_results=5)
        ddg_tool = DuckDuckGoSearchRun(
            api_wrapper=ddg_wrapper,
            description="Search the web using DuckDuckGo for current information and news."
        )
        tools_list.append(ddg_tool)
    except Exception as e:
        print(f"Note: DuckDuckGo search not available due to error: {e}")
    
    # Add custom tools
    tools_list.extend([
        calculator,
        code_analyzer,
        weather_info,
        file_content_generator
    ])
    
    return tools_list



# LLM WITH MULTIPLE MODELS


In [4]:

class EnhancedLLM:
    """Enhanced LLM class with fallback models and error handling"""
    
    def __init__(self):
        self.primary_model = "llama-3.3-70b-versatile"  # Fast, capable model
        self.fallback_models = [
            "llama-3.2-90b-text-preview",  # More capable for complex tasks
            "mixtral-8x7b-32768",          # Good for reasoning
            "gemma2-9b-it"                 # Lightweight fallback
        ]
        self.current_model = self.primary_model
        
    def get_llm(self, temperature: float = 0.1, max_tokens: int = 2000):
        """Get LLM instance with current model"""
        try:
            return ChatGroq(
                model=self.current_model,
                temperature=temperature,
                max_tokens=max_tokens
            )
        except Exception as e:
            print(f"Error with model {self.current_model}: {e}")
            # Try fallback models
            for model in self.fallback_models:
                try:
                    self.current_model = model
                    return ChatGroq(
                        model=model,
                        temperature=temperature,
                        max_tokens=max_tokens
                    )
                except:
                    continue
            raise Exception("All models failed to initialize")

# ================================================================================
# ENHANCED NODES WITH ERROR HANDLING AND CONTEXT
# ================================================================================

def create_enhanced_nodes(tools: List, llm_manager: EnhancedLLM):
    """Create enhanced nodes with better error handling and context management"""
    
    def context_aware_llm(state: ConversationState) -> ConversationState:
        """Enhanced LLM node with context awareness and error handling"""
        try:
            # Get current LLM
            llm = llm_manager.get_llm()
            llm_with_tools = llm.bind_tools(tools=tools)
            
            # Add system message with context if this is the first message
            messages = state["messages"]
            if not any(isinstance(msg, SystemMessage) for msg in messages):
                system_prompt = f"""You are an advanced AI assistant with access to multiple tools. 
Current conversation summary: {state.get('conversation_summary', 'New conversation')}
User context: {state.get('user_context', {})}
Last successful tool: {state.get('last_tool_used', 'None')}

Guidelines:
1. Use tools when they can provide better, more current, or specialized information
2. Be concise but comprehensive in your responses  
3. If a tool fails, try alternative approaches
4. Maintain conversation context and refer to previous interactions when relevant
5. For complex queries, break them down and use multiple tools if needed"""
                
                messages = [SystemMessage(content=system_prompt)] + messages
            
            # Invoke LLM with tools
            response = llm_with_tools.invoke(messages)
            
            # Update state
            new_state = {
                "messages": [response],
                "error_count": 0  # Reset error count on success
            }
            
            return new_state
            
        except Exception as e:
            error_msg = f"Error in LLM processing: {str(e)}"
            print(error_msg)
            
            # Increment error count
            error_count = state.get("error_count", 0) + 1
            
            # Fallback response
            fallback_response = AIMessage(
                content=f"I encountered an issue processing your request. Error count: {error_count}. "
                       f"Let me try a different approach or please rephrase your question."
            )
            
            return {
                "messages": [fallback_response],
                "error_count": error_count
            }
    
    def enhanced_tool_node(state: ConversationState) -> ConversationState:
        """Enhanced tool node with caching and error handling"""
        try:
            # Use standard ToolNode but with enhanced error handling
            tool_node = ToolNode(tools)
            result = tool_node.invoke(state)
            
            # Cache successful tool results
            if "tool_results_cache" not in state:
                state["tool_results_cache"] = {}
            
            # Update last successful tool
            last_message = result["messages"][-1] if result["messages"] else None
            if last_message and hasattr(last_message, 'name'):
                result["last_tool_used"] = last_message.name
            
            return result
            
        except Exception as e:
            error_msg = f"Tool execution error: {str(e)}"
            print(error_msg)
            
            # Return error message
            error_response = AIMessage(
                content=f"I encountered an error while using the tools: {str(e)}. "
                       "Let me try to help you in a different way."
            )
            
            return {"messages": [error_response]}
    
    return context_aware_llm, enhanced_tool_node



# CONVERSATION MANAGER


In [5]:
# ================================================================================
# CONVERSATION MANAGER
# ================================================================================

class ConversationManager:
    """Manages conversation history and context"""
    
    def __init__(self, max_history: int = 20):
        self.max_history = max_history
        
    def summarize_conversation(self, messages: List[AnyMessage]) -> str:
        """Create a summary of the conversation"""
        if len(messages) < 4:
            return "New conversation"
        
        # Simple summarization - count topics discussed
        topics = set()
        for msg in messages:
            if isinstance(msg, HumanMessage):
                content = msg.content.lower()
                if any(word in content for word in ['weather', 'temperature', 'climate']):
                    topics.add('weather')
                if any(word in content for word in ['calculate', 'math', 'equation']):
                    topics.add('calculations')
                if any(word in content for word in ['code', 'programming', 'python']):
                    topics.add('programming')
                if any(word in content for word in ['research', 'paper', 'study']):
                    topics.add('research')
        
        if topics:
            return f"Discussion topics: {', '.join(topics)}"
        return f"Conversation with {len(messages)//2} exchanges"
    
    def trim_history(self, messages: List[AnyMessage]) -> List[AnyMessage]:
        """Trim conversation history to manageable size"""
        if len(messages) <= self.max_history:
            return messages
        
        # Keep system message if present, plus recent messages
        system_msgs = [msg for msg in messages if isinstance(msg, SystemMessage)]
        recent_msgs = messages[-self.max_history:]
        
        return system_msgs + recent_msgs

# ================================================================================
# MAIN CHATBOT CLASS
# ================================================================================

class AdvancedChatbot:
    """Advanced chatbot with multiple tools and enhanced features"""
    
    def __init__(self):
        self.tools = initialize_tools()
        self.llm_manager = EnhancedLLM()
        self.conversation_manager = ConversationManager()
        self.memory = MemorySaver()
        self.graph = self._build_graph()
        
        print(f"✅ Initialized chatbot with {len(self.tools)} tools:")
        for tool in self.tools:
            print(f"   - {tool.name}: {tool.description}")
    
    def _build_graph(self) -> StateGraph:
        """Build the enhanced conversation graph"""
        
        # Create enhanced nodes
        context_llm, enhanced_tools = create_enhanced_nodes(self.tools, self.llm_manager)
        
        # Build graph
        builder = StateGraph(ConversationState)
        
        # Add nodes
        builder.add_node("context_llm", context_llm)
        builder.add_node("enhanced_tools", enhanced_tools)
        builder.add_node("conversation_manager", self._manage_conversation)
        
        # Add edges
        builder.add_edge(START, "conversation_manager")
        builder.add_edge("conversation_manager", "context_llm")
        
        # Conditional edge from LLM
        builder.add_conditional_edges(
            "context_llm",
            tools_condition,
            {
                "tools": "enhanced_tools",
                "__end__": END
            }
        )
        
        # Edge from tools back to LLM
        builder.add_edge("enhanced_tools", "context_llm")
        
        # Compile with memory
        return builder.compile(checkpointer=self.memory)
    
    def _manage_conversation(self, state: ConversationState) -> ConversationState:
        """Manage conversation context and history"""
        
        # Trim history if needed
        messages = self.conversation_manager.trim_history(state["messages"])
        
        # Update conversation summary
        summary = self.conversation_manager.summarize_conversation(messages)
        
        # Initialize user context if not present
        user_context = state.get("user_context", {})
        
        return {
            "messages": messages,
            "conversation_summary": summary,
            "user_context": user_context,
            "tool_results_cache": state.get("tool_results_cache", {}),
            "error_count": state.get("error_count", 0),
            "last_tool_used": state.get("last_tool_used", "")
        }
    
    def chat(self, message: str, thread_id: str = "default") -> str:
        """Main chat interface"""
        try:
            # Create initial state
            initial_state = {
                "messages": [HumanMessage(content=message)],
                "user_context": {},
                "tool_results_cache": {},
                "conversation_summary": "",
                "error_count": 0,
                "last_tool_used": ""
            }
            
            # Run the graph
            config = {"configurable": {"thread_id": thread_id}}
            result = self.graph.invoke(initial_state, config)
            
            # Extract final response
            final_messages = result["messages"]
            ai_messages = [msg for msg in final_messages if isinstance(msg, AIMessage)]
            
            if ai_messages:
                return ai_messages[-1].content
            else:
                return "I'm sorry, I couldn't generate a response. Please try again."
                
        except Exception as e:
            return f"Error in chat processing: {str(e)}. Please try rephrasing your question."
    
    def get_conversation_history(self, thread_id: str = "default") -> List[Dict]:
        """Get conversation history for a thread"""
        try:
            config = {"configurable": {"thread_id": thread_id}}
            history = []
            
            for state in self.graph.get_state_history(config):
                messages = state.values.get("messages", [])
                for msg in messages:
                    if isinstance(msg, (HumanMessage, AIMessage)):
                        history.append({
                            "type": "human" if isinstance(msg, HumanMessage) else "ai",
                            "content": msg.content,
                            "timestamp": datetime.now().isoformat()
                        })
            
            return history
        except:
            return []



# EXAMPLE USAGE AND TESTING


In [6]:
# ================================================================================
# ================================================================================

def run_examples():
    """Run example conversations to demonstrate capabilities"""
    
    # Initialize chatbot
    chatbot = AdvancedChatbot()
    
    # Example conversations
    examples = [
        "Calculate the area of a circle with radius 5",
        "What's the latest research on quantum computing?",
        "Analyze this Python code: def hello(): print('Hello World')",
        "What's the weather like in Tokyo?",
        "Generate a CSV file for student grades",
        "Who is Nikola Tesla?",
        "Find recent news about artificial intelligence in healthcare"
    ]
    
    print("\n" + "="*60)
    print("ADVANCED CHATBOT DEMONSTRATION")
    print("="*60)
    
    for i, example in enumerate(examples, 1):
        print(f"\n🤖 Example {i}: {example}")
        print("-" * 50)
        
        try:
            response = chatbot.chat(example, thread_id=f"demo_{i}")
            print(f"✅ Response: {response}")
        except Exception as e:
            print(f"❌ Error: {str(e)}")
        
        print("\n" + "-" * 50)



# JUPYTER NOTEBOOK INTEGRATION


In [9]:

def display_chatbot_interface():
    """Display an interactive interface for Jupyter"""
    from IPython.display import display, HTML, clear_output
    import ipywidgets as widgets
    
    # Initialize chatbot
    chatbot = AdvancedChatbot()
    
    # Create output with scroll + full width
    # output = widgets.Output(layout=widgets.Layout(
    #     width='80%',
    #     height='400px',
    #     overflow='auto',
    #     border='1px solid #ccc',
    #     padding='10px',
    #    # white_space='normal'
    # ))
    output= widgets.VBox(layout=widgets.Layout(
    width='100%',
    height='400px',
    overflow_y='auto',
    border='1px solid #ccc',
    padding='10px',
))


    
    # Input widget
    text_input = widgets.Text(
        placeholder="Type your message here...",
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='100%')  # changed from 70%
    )
    
    # Buttons
    send_button = widgets.Button(
        description="Send",
        button_style='primary',
        layout=widgets.Layout(width='100%')
    )
    clear_button = widgets.Button(
        description="Clear",
        button_style='warning',
        layout=widgets.Layout(width='100%')
    )
    
    # Button container (horizontal)
    button_box = widgets.HBox([send_button, clear_button],
                               layout=widgets.Layout(justify_content='space-between'))

    # Chat history
    chat_history = []
    

    # def send_message(b):
        # message = text_input.value.strip()
        # if message:
        #     with output:
        #         print(f"🧑 You: {message}")
        #         try:
        #             response = chatbot.chat(message)
        #             from IPython.display import HTML as ipyHTML, display as ipy_display
        #             ipy_display(ipyHTML(f"<div style='white-space: normal; word-wrap: break-word;'>🤖 Assistant: {response}</div>"))
        #             chat_history.append(("user", message))
        #             chat_history.append(("assistant", response))
        #         except Exception as e:
        #             print(f"❌ Error: {str(e)}")
        #         print("\n" + "-" * 80 + "\n")
        #     text_input.value = ""



    # def send_message(b):
    #     message = text_input.value.strip()
    #     if message:
    #         user_html = widgets.HTML(
    #             value=f"<div style='white-space: normal; word-wrap: break-word;'><b>🧑 You:</b> {message}</div>"
    #         )
    #         try:
    #             response = chatbot.chat(message)
    #             assistant_html = widgets.HTML(
    #                 value=f"<div style='white-space: normal; word-wrap: break-word;'><b>🤖 Assistant:</b> {response}</div>"
    #             )
    #         except Exception as e:
    #             assistant_html = widgets.HTML(
    #                 value=f"<div style='color: red;'>❌ Error: {str(e)}</div>"
    #             )
    #         output.children += (user_html, assistant_html)
    #         chat_history.append(("user", message))
    #         chat_history.append(("assistant", response))
    #         text_input.value = ""

        ## adding format_response function to format the response

         
    def format_response(response_text):
        """Converts bullet points and newlines to HTML format."""
        response_text = html.escape(response_text)
        lines = response_text.split("\n")
        html_lines = []
        in_list = False
        for line in lines:
            if line.strip().startswith("* "):
                if not in_list:
                    html_lines.append("<ul>")
                    in_list = True
                html_lines.append(f"<li>{line.strip()[2:]}</li>")
            else:
                if in_list:
                    html_lines.append("</ul>")
                    in_list = False
                html_lines.append(f"<p>{line}</p>")
        if in_list:
            html_lines.append("</ul>")
        return "<div style='white-space: normal; word-wrap: break-word;'>" + "".join(html_lines) + "</div>"
    
    def send_message(b):
        message = text_input.value.strip()
        if message:
            user_html = widgets.HTML(
                value=f"<div style='white-space: normal; word-wrap: break-word;'><b>🧑 You:</b> {html.escape(message)}</div>"
            )
            try:
                response = chatbot.chat(message)
                formatted_response = format_response(response)
                assistant_html = widgets.HTML(
                    value=f"<b>🤖 Assistant:</b> {formatted_response}"
                )
            except Exception as e:
                assistant_html = widgets.HTML(
                    value=f"<div style='color: red;'>❌ Error: {str(e)}</div>"
                )
            output.children += (user_html, assistant_html)
            chat_history.append(("user", message))
            chat_history.append(("assistant", response))
            text_input.value = ""



    
    # def clear_chat(b):
    #     with output:
    #         clear_output()
    #         chat_history.clear()
    #         print("Chat cleared. Start a new conversation!")

    def clear_chat(b):
        output.children = ()  # Clear all displayed chat messages
        chat_history.clear()
        print("Chat cleared. Start a new conversation!")

    
    # Event handlers
    send_button.on_click(send_message)
    clear_button.on_click(clear_chat)
    text_input.on_submit(lambda x: send_message(None))
    
    # Layout containers
    input_area = widgets.VBox([
        text_input,
        button_box
    ], layout=widgets.Layout(width='100%'))

    full_ui = widgets.VBox([output, input_area],
                           layout=widgets.Layout(width='100%', overflow_x='auto'))
    
    # Display
    print("🚀 Advanced Multi-Tool Chatbot Interface")
    print("Available tools: ArXiv, Wikipedia, Web Search, Calculator, Code Analyzer, Weather, File Generator")
    print("Type your message and press Enter or click Send!")
    print("-" * 80)
    
    display(full_ui)
    
    return chatbot


# Example usage in Jupyter
if __name__ == "__main__":
    # For notebook environment
    try:
        get_ipython()  # This will raise NameError if not in Jupyter
        print("🎯 Run display_chatbot_interface() to start the interactive chat!")
        print("🎯 Run run_examples() to see demonstration examples!")
    except NameError:
        # For script environment
        run_examples()

🎯 Run display_chatbot_interface() to start the interactive chat!
🎯 Run run_examples() to see demonstration examples!


In [10]:
display_chatbot_interface()

✅ Initialized chatbot with 8 tools:
   - arxiv: Search academic papers on arXiv. Best for recent research papers and preprints.
   - wikipedia: Search Wikipedia for general knowledge, definitions, and factual information.
   - tavily_search_results_json: Search the web for recent news, current events, and real-time information.
   - duckduckgo_search: Search the web using DuckDuckGo for current information and news.
   - calculator: Perform mathematical calculations safely.

Args:
    expression: Mathematical expression to evaluate (e.g., "2 + 3 * 4")

Returns:
    Result of the calculation or error message
   - code_analyzer: Analyze code for basic syntax and provide simple feedback.

Args:
    code: Code snippet to analyze
    language: Programming language (default: python)

Returns:
    Basic analysis of the code
   - weather_info: Get weather information for a location (simulated for demo).

Args:
    location: City or location name

Returns:
    Weather information
   - file_cont

C:\Users\Waseek\AppData\Local\Temp\ipykernel_9048\410335709.py:156: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  text_input.on_submit(lambda x: send_message(None))
